In [ ]:
pip install PyMuPDF

In [ ]:
pip install BERTopic

In [ ]:
import os
import fitz
import nltk
import re
import string
import gensim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pprint


# For tokenization
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download('wordnet')

# For removing stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')

# For lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

# For TF-IDC
from gensim import corpora
from gensim.models import TfidfModel

# For N-grams
from collections import defaultdict
from nltk.util import ngrams as nltk_ngrams
from nltk.tokenize import word_tokenize

# For BERT topic
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# For coherence score
from gensim.models.coherencemodel import CoherenceModel

### Get text from PDFs

In [4]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf.load_page(page_num)
            text += page.get_text()
    return text

### Preprocess and vectorize text

*Preprocess pipeline*

In [5]:
# Loading WordNet POS tags for lemmatization
def wordnet_pos_tags(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [34]:
def txt_preprocess_pipeline(text):

    # Removing multiple white spaces and line breaks
    clean_txt = re.sub(r'\n', ' ', text)
    clean_txt = re.sub(r'\s+', ' ', clean_txt)
    clean_txt = clean_txt.strip()

    # Tokenizing text
    tokens = nltk.word_tokenize(clean_txt)

    # Loading NLTK stopword list and adding original stopwords
    stop_words = set(stopwords.words('english')+['impact','event','area','project','region','change','country','na','na na','city','oecd','urban','red','paper','forum','figgs','enfield','literature','estimate','model','effect','impact','extreme','study','tantly','treme','tsuji','tural','yt','en','ings','factsheet','edney','budin','apps','app','enbel','alth','aunan','basu','user','search','photo','tures','mistry','letter','cdds','ly','national','plan','hhaps','use','initiate','various','public','health','heat','temperature','change','risk', 'increase','health','india','plan', 'brutally','management','part','highlight','world','united','nation','joint', 'cooperation','brief','issue','professor','tsdps','ons','qlw','lfb','qh','qc','humber','osdma','hse','hm','maha','ucs','lok','international','doya','gsdma','cdri','eswd','dluhc','esd','dfe','niosh','naic','desnz','odisha','defra','easac','bouhi','awhp','ndma','rachel','bureau','scientist','adequate','annual','address','amplify','billion','calculation,','activate','adaptation','additionally','acknowledge','afternoon','accordance','act','accord','data','high','ukhsa','osha','mrt','donthu','ensemble','kristina','webinar','qr','percent','interviewee','licker','eea','ccc','conv','january','iiphb','ferguson','ulbrich','trenberth','schär','rcms','nao','germi','veisz','spell','sres','srex','phenomenon','probability','ρ','qsw','moj','nhs','imd','october','academicians','academic','absorb','aapda','nrdc','report','online','across','action','expand','page','u','https','j','g','figure','thank','you','mbove','print','info','escalate','isbn','volume','dr','vol','phd','upon','al', 'e', 'fig', 'cl', 'hap', 'et', 'http', 'ew', 'climate','jun','simon', 'yang','go','copyright','ltd','license','creative','imply','dot', 'dashed','line', 'may', 'either','bi','de', 'la', 'paix', 'tel', 'box', 'fax','front', 'oli', 'scarff', 'getty', 'image', 'back', 'atlas','ness','net', 'san', 'felipe','also','informa', 'tion', 'per', 'cent','due','distri', 'bution','com','bin','thus','km', 'grid','would','christopher','u','aa','aap','aas','ab','abares','abatzoglou','abbs','abc','aberystwyth','doi'])

    # Adding custom stopwords directly to the set
    custom_stopwords = set(['impact','event','area','project','region','change','country','na','na na','city','oecd','urban','red','paper','forum','figgs','enfield','literature','estimate','model','effect','impact','extreme','study','tantly','treme','tsuji','tural','yt','en','ings','factsheet','edney','budin','apps','app','enbel','alth','aunan','basu','user','search','photo','tures','mistry','letter','cdds','ly','national','plan','hhaps','use','initiate','various','public','health', 'heat', 'temperature', 'change', 'risk', 'increase','health','india','plan', 'brutally','management','part','highlight','world','united','nation','joint', 'cooperation','brief','issue','professor','tsdps','ons','qlw','lfb','qh','qc','humber','osdma','hse','hm','maha','ucs','lok','international','doya','gsdma','cdri','eswd','dluhc','esd','dfe','niosh','naic','desnz','odisha','defra','easac','bouhi','awhp','ndma','rachel','bureau','scientist','adequate','annual','address','amplify','billion','calculation,','activate','adaptation','additionally','acknowledge','afternoon','accordance','act','accord','data','high','ukhsa','osha','mrt','donthu','ensemble','kristina','webinar','qr','percent','interviewee','licker','eea','ccc','conv','january','iiphb','ferguson','ulbrich','trenberth','schär','rcms','nao','germi','veisz','spell','sres','srex','phenomenon','probability','ρ','qsw','moj','nhs','imd','october','academicians','academic','absorb','aapda','nrdc','report','online','across','action','expand','page','u','https','j','g','figure','thank','you','mbove','print','info','escalate','isbn','volume','dr','vol','phd','upon','al', 'e', 'fig', 'cl', 'hap', 'et', 'http', 'ew', 'climate','jun','simon', 'yang','go','copyright','ltd','license','creative','imply','dot', 'dashed','line', 'may', 'either','bi','de', 'la', 'paix', 'tel', 'box', 'fax','front', 'oli', 'scarff', 'getty', 'image', 'back', 'atlas','ness','net', 'san', 'felipe','also','informa', 'tion', 'per', 'cent','due','distri', 'bution','com','bin','thus','km', 'grid','would','christopher','u','aa','aap','aas','ab','abares','abatzoglou','abbs','abc','aberystwyth','doi'])
    stop_words.update(custom_stopwords)

    # Standardize the text and remove non-alphabetic or stop words
    words = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]

    # Defining lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Conducting POS tagging
    pos_tags = nltk.pos_tag(words)

    # Lemmatizing word-tokens via assigned POS tags
    lemma_tokens = [lemmatizer.lemmatize(token, wordnet_pos_tags(pos_tag)) for token, pos_tag in pos_tags]

    return lemma_tokens

*Iteration function*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def iterate_pdf_files(pdf_folder):
    # Creating a dictionary to display all documents and tokens together
    docs = {}

    # Counting to tracking the number of processed texts
    text_count = 0

    # Iterating over each file in the folder
    for filename in os.listdir(pdf_folder):

        # Checking whether the files are PDFs
        if filename.endswith('.pdf'):

            # Creating a path to the PDF files
            pdf_path = os.path.join(pdf_folder, filename)

            # Extracting the content of files
            text = extract_text_from_pdf(pdf_path)

            # Applying the preprocessing pipeline to the extracted tex
            preprocessed_text = txt_preprocess_pipeline(text)

            # Generating a unique name for the document based on the length of the docs dictionary for ease in readibility
            doc_name = f'doc{len(docs) + 1}'

            # Assigning the preprocessed text content of the current document in the docs dictionary with the document name as the key
            docs[doc_name] = [preprocessed_text]

            # Increasing the text count when finished assigning
            text_count += 1

    return docs

pdf_folder = '/content/drive/MyDrive/risk'

docs = iterate_pdf_files(pdf_folder)

# Printing each document's tokens as a list
for doc_name, tokens in docs.items():
    print(f"{doc_name}: {tokens}")

*Token dictionary*

In [ ]:
# Iterating over the values in the docs dictionary created in the previous step. Each value in the sublist represents the tokens in a certain document.
all_preprocessed_text = [token for sublist in docs.values() for token in sublist]

# Creating a gensim dictionary to assign an integer ID to each token
dictionary = corpora.Dictionary(all_preprocessed_text)
print(dictionary)

*BoW representation*

In [ ]:
# Creating a Bag-of-Words (BoW) representation of the preprocessed text data using the dictionary created in the previous step
bow_corpus = [dictionary.doc2bow(text) for text in all_preprocessed_text]
pprint.pprint(bow_corpus)

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(bow_corpus))

### TF-IDF

In [ ]:
# Creating a TF-IDF model from the Bag-of-Words corpus
tfidf_model = TfidfModel(bow_corpus)

# Applying the TF-IDF model to the Bag-of-Words corpus to get the TF-IDF representation
tfidf_corpus = tfidf_model[bow_corpus]

# Printing the TF-IDF representation for the first few documents
for doc in tfidf_corpus[:5]:
    print(doc)


In [ ]:
# Looking up the word corresponding to word_id = 0
word_id = 5
word = dictionary[word_id]
print("Word corresponding to word_id =", word_id, ":", word)

In [ ]:
# Defining the TF-IDF threshold
threshold = 0.05

# Iterating through the TF-IDF representation of the documents
for doc in tfidf_corpus[:5]:  # Iterating over the first few documents

    # Filtering words with TF-IDF scores above the threshold
    words_above_threshold = [(dictionary[word_id], tfidf_score) for word_id, tfidf_score in doc if tfidf_score > threshold]

    # Printing the words with TF-IDF scores above the threshold
    print("Words with TF-IDF score > 0.05:", words_above_threshold)

### N-grams

In [ ]:
# Taking a list of tokens as input and generating n-grams
def generate_ngrams(tokens, n=2):
    return list(nltk_ngrams(tokens, n))

# Creating n-gram model from a corpus
def create_ngram_model(corpus, n=2):
    ngram_model = defaultdict(int)
    for text in corpus:
        tokens = word_tokenize(text)
        ngrams = generate_ngrams(tokens, n)
        for ngram in ngrams:
            ngram_model[ngram] += 1
    return ngram_model

max_texts_to_process = None
docs = iterate_pdf_files(pdf_folder)

# Concatenating preprocessed tokens for each document into a single list of tokens
corpus_tokens = [tokens for sublist in docs.values() for tokens in sublist]

# Concatenating tokens into a single string for each document
corpus_strings = [' '.join(tokens) for tokens in corpus_tokens]

# Creating an n-gram model
ngram_model = create_ngram_model(corpus_strings, n=2)

# Printing all n-grams
print("All N-grams:")
for ngram, frequency in ngram_model.items():
    print(ngram, ":", frequency)

### BERT topic

In [36]:
def preprocess_texts(docs):
    preprocessed_docs = [' '.join(tokens) for tokens in (doc[0] for doc in docs.values())]
    return preprocessed_docs
preprocessed_texts = preprocess_texts(docs)

In [ ]:
# Initializing the SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encoding the preprocessed texts into embeddings
embeddings = model.encode(preprocessed_texts, show_progress_bar=True)

In [44]:
# Initializing BERTopic with custom parameters
topic_model = BERTopic(min_topic_size=2,
                       top_n_words=5,
                       n_gram_range=(1, 2),
                       calculate_probabilities=True)

# Fitting BERTopic on the preprocessed texts and embeddings
topics, probs = topic_model.fit_transform(preprocessed_texts,embeddings)

In [45]:
# Checking the topics identified
topic_info = topic_model.get_topic_info()
print(topic_info)

    Topic  Count                                               Name  \
0      -1     11                 -1_australia_water_australian_warm   
1       0     31                       0_flood_disaster_impact_area   
2       1     11                             1_ehe_green_water_cool   
3       2     11               2_change_project_water_precipitation   
4       3     10                         3_drought_water_event_mean   
5       4      9               4_queensland_fire_heatwave_emergency   
6       5      7                  5_forecast_mortality_cool_weather   
7       6      6  6_anomaly_precipitation_attribution_anthropogenic   
8       7      6                     7_heatwave_heatwaves_trend_day   
9       8      5                              8_utci_ta_tibet_trend   
10      9      5                  9_fishery_chwesl_chwesl event_sea   
11     10      3              10_mortality_mortality rate_rate_ewes   

                                       Representation  \
0        [australia

In [60]:
topic_model.get_topic(4)

[('queensland', 0.033258904829994565),
 ('fire', 0.028118352994422172),
 ('heatwave', 0.02370616205229062),
 ('emergency', 0.01919691771492977),
 ('bushfire', 0.0190237409236173)]

In [46]:
# Getting the topic words
topic_words = topic_model.get_topics()

# Preprocessing the texts
preprocessed_texts = preprocess_texts(docs)

# Converting preprocessed texts to list of words for coherence model
texts = [doc.split() for doc in preprocessed_texts]

# Creating a dictionary and corpus for the coherence model
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

coherence_scores = []
for topic_id, topic in topic_words.items():
    # Preparing the data for coherence model
    topics = [[word for word, _ in topic]]

    # Computing Coherence Score using Gensim's CoherenceModel for each topic
    coherence_model = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    coherence_scores.append((topic_id, coherence_score))

# Printing coherence scores for each topic
for topic_id, coherence_score in coherence_scores:
    print(f'Topic {topic_id}: Coherence Score: {coherence_score}')

Topic -1: Coherence Score: 0.5961401002909599
Topic 0: Coherence Score: 0.5732218838737833
Topic 1: Coherence Score: 0.637400711843347
Topic 2: Coherence Score: 0.6176852614555182
Topic 3: Coherence Score: 0.5656932913561492
Topic 4: Coherence Score: 0.8032704067448737
Topic 5: Coherence Score: 0.5528102746404895
Topic 6: Coherence Score: 0.6779725467346835
Topic 7: Coherence Score: 0.6630834536465461
Topic 8: Coherence Score: 0.5155617820027103
Topic 9: Coherence Score: 0.4692130955977549
Topic 10: Coherence Score: 0.7224043979440935
